In [67]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import requests

# Select the representative profile

Considering the following parameters
- tipo de candidatura = Diputación mayoría relativa 
- entidad = all states, iterate through hard-coded list
-

Open driver and webpage

In [68]:
# Set up WebDriver (assuming you're using Chrome)
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": "C:/Users/luisf/Downloads/propuestas_politicas",  # Change path accordingly
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

driver = webdriver.Chrome(options=options)

driver.get('https://candidaturas2021.ine.mx/')

Select tipo de candidatura

In [69]:
# Give it a moment to load options
time.sleep(5)

# select only those that correspond to the tipo_de_candidatura == mayoria relativa
tipo_candidatura = driver.find_element(By.CSS_SELECTOR, '#busqueda-form_tipoCandidatura')
# Click the dropdown to reveal the options
tipo_candidatura.click()

# select the needed title
mayoria_relativa = driver.find_element(By.CSS_SELECTOR, 'div[title="Diputación mayoría relativa"]')
mayoria_relativa.click()

Select state

In [70]:
# select only those that correspond to the tipo_de_candidatura == mayoria relativa
entidad = driver.find_element(By.CSS_SELECTOR, '#busqueda-form_entidad')
# Click the dropdown to reveal the options
entidad.click()

# select the needed state
nombre_estado = "AGUASCALIENTES"
estado = driver.find_element(By.CSS_SELECTOR, f'div[title="{nombre_estado}"]')
estado.click()

# Give it a moment to load options
time.sleep(2)

Execute search

In [ ]:
#execute search
driver.execute_script("window.scrollBy(0, 500);")  # Scroll down by 1000 pixels
buscar = driver.find_element(By.CSS_SELECTOR, 'button.ant-btn:nth-child(2)')
# Click the dropdown to reveal the options
time.sleep(1)
buscar.click()
time.sleep(2)

Table selection

In [104]:

# select only those that correspond to the tipo_de_candidatura == mayoria relativa
tabla = driver.find_element(By.CSS_SELECTOR, '#contenedor-resultados > div.ant-table-wrapper > div > div > div > div > div > table')

# Find all rows in the table (usually inside <tr> tags)
rows = tabla.find_elements(By.XPATH, './/tbody/tr')    

In [84]:
# Loop through each row and find the url with the file to download
n=1
for row in range(1,2):#range(1,len(rows)): #the table links begin on the 2nd body row, get the link and open, download file, and go back
    n=n+1
    tabla = driver.find_element(By.CSS_SELECTOR, '#contenedor-resultados > div.ant-table-wrapper > div > div > div > div > div > table')
    rows = tabla.find_elements(By.XPATH, './/tbody/tr')

    # Locate the 4th column (index 3 because it is 0-based)
    link_element = tabla.find_element(By.CSS_SELECTOR, f'tr.ant-table-row:nth-child({n}) > td:nth-child(4) > div:nth-child(1) > a:nth-child(2)')
    # Extract the link and print it (optional)
    link_url = link_element.get_attribute('href')
    print(f"Opening link: {link_url}")

    # Click the link
    link_element.click()
    # Allow the new page to load (adjust time as necessary)
    time.sleep(2)
    driver.execute_script("window.scrollBy(0, 5000);")  # Scroll down by 1000 pixels

    documento = driver.find_element(By.CSS_SELECTOR, '#imprimir > div.ant-row > div:nth-child(2) > div > a')
    documento.click()
    # Optionally, you can perform some action on the new page here
    
    # Go back to the original table page
    driver.back()
    time.sleep(4)


<selenium.webdriver.remote.webelement.WebElement (session="e3e225035c7b02461aef9bf37492c032", element="f.A536FCADE710D4A325F2347AA880DCC0.d.37C6CE1EE3B2D23D4C1741C29A520490.e.143")>
<selenium.webdriver.remote.webelement.WebElement (session="e3e225035c7b02461aef9bf37492c032", element="f.A536FCADE710D4A325F2347AA880DCC0.d.37C6CE1EE3B2D23D4C1741C29A520490.e.144")>
<selenium.webdriver.remote.webelement.WebElement (session="e3e225035c7b02461aef9bf37492c032", element="f.A536FCADE710D4A325F2347AA880DCC0.d.37C6CE1EE3B2D23D4C1741C29A520490.e.145")>
<selenium.webdriver.remote.webelement.WebElement (session="e3e225035c7b02461aef9bf37492c032", element="f.A536FCADE710D4A325F2347AA880DCC0.d.37C6CE1EE3B2D23D4C1741C29A520490.e.146")>
<selenium.webdriver.remote.webelement.WebElement (session="e3e225035c7b02461aef9bf37492c032", element="f.A536FCADE710D4A325F2347AA880DCC0.d.37C6CE1EE3B2D23D4C1741C29A520490.e.147")>
<selenium.webdriver.remote.webelement.WebElement (session="e3e225035c7b02461aef9bf37492c03

In [108]:
driver.execute_script("window.scrollBy(0, 2500);")  # Scroll down by 1000 pixels


In [72]:


# Give it a moment to load options
time.sleep(2)


# Close the browser after the task is done
#driver.quit()